In [1]:
using Laplacians

### first gaus siedel and jacobi - these seem to be pretty unefficient

In [2]:
include("../../../src/preconditioners.jl")

gs (generic function with 1 method)

In [3]:
n = 10000
a = chimera(n,1);
la = lap(a)
b = rand(n); b = b - mean(b);

In [4]:
f = jacobi(la);

In [5]:
@time x = pcg(la, b, f, tol = 1e-2, maxits = 10000, verbose=true);

PCG stopped after: 5718 iterations with relative error 0.009988125014519286.
  2.734116 seconds (723.87 k allocations: 2.157 GB, 9.55% gc time)


In [6]:
norm(la * x - b)

0.2860247687542121

### let's take a look at the algebraic multigrid solver

In [7]:
using PyAMG

In [8]:
@time x = PyAMG.solve(la, b, tol = 1e-7, maxiter = 100);

  3.217712 seconds (1.09 M allocations: 46.501 MB, 0.48% gc time)
  Detected a Hermitian matrix
    maxiter = 100
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
    iteration 8
    iteration 9
    iteration 10
    iteration 11
    iteration 12
    iteration 13
    iteration 14
    iteration 15
  Residual reduction ||r_k||/||r_0|| = 8.59e-08


/Users/serbanstan/.julia/v0.4/Conda/deps/usr/lib/python2.7/site-packages/pyamg/strength.py:533: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if B == 'ones':


In [9]:
norm(la * x - b)

7.526295014062325e-6

In [10]:
?PyAMG.solve

`solve(amg::AMGSolver, b, kwargs...)`

Returns a `Vector` with the result of the AMG solver.

### `kwargs`  (copy-pasted from Python docs)

  * `x0` : Initial guess.
  * `tol` : Stopping criteria: relative residual r[k]/r[0] tolerance.
  * `maxiter` : Stopping criteria: maximum number of allowable iterations.
  * `cycle` : {"V","W","F","AMLI"}     Type of multigrid cycle to perform in each iteration.
  * `accel` : Defines acceleration method.  Can be a string such as "cg"     or "gmres" which is the name of an iterative solver in     `pyamg.krylov` (preferred) or scipy.sparse.linalg.isolve.     If accel is not a string, it will be treated like a function     with the same interface provided by the iterative solvers in SciPy.         (the function version is not tested in Julia!)
  * `callback` : User-defined function called after each iteration.  It is     called as callback(xk) where xk is the k-th iterate vector.
  * `residuals` : List to contain residual norms at each iteration.

`solve(A::SparseMatrixCSC, b::Vector; kwargs...)`:

PyAMG's 'blackbox' solver. See `pyamg.solve?` for `kwargs`.


### now, let's test on harder graphs

In [13]:
a = readFromFile("3.txt");

In [14]:
n = a.n

100000

In [15]:
la = lap(a);
b = rand(n); b = b - mean(b);

In [16]:
@time x = PyAMG.solve(la, b, tol = 1e-6, maxiter = 100);

 60.086554 seconds (221 allocations: 14.667 MB)
  Detected a Hermitian matrix
    maxiter = 100
    iteration 1
    iteration 2
    iteration 3
    iteration 4
    iteration 5
    iteration 6
    iteration 7
  Residual reduction ||r_k||/||r_0|| = 7.63e-08


In [17]:
norm(la * x - b)

0.00013116309157574173

In [18]:
include("../../../src/samplingTreeSolver.jl");

In [21]:
@time f = samplingSolver(a, tol = 1e-6, maxits = 1000, eps = 0.5, sampConst = 0.02, beta = 1000.0);

rho = 10.603796220956797
Average number of multiedges = 2.8483223868531704
Maximum number of multiedges = 10.603796220957246
Time to build the tree and compute the stretch: elapsed time: 1.462410671 seconds
  3.346280 seconds (21.51 M allocations: 1.290 GB, 14.45% gc time)


In [23]:
@time norm(la * f(b) - b)

  4.827660 seconds (48.00 M allocations: 1.793 GB, 5.79% gc time)


9.067297933684568e-5

### so, we solve in around 9 seconds.